In [1]:
import time
import json
import nest_asyncio
import asyncio
import websockets
import numpy as np
import math
import random
import colorsys
from tqdm.notebook import tqdm
nest_asyncio.apply()

In [2]:
async def websocket_test(payload, indent=None, quiet=False):
    uri = "ws://127.0.0.1:8765/ws"
    async with websockets.connect(uri) as ws:
        await ws.send(json.dumps(payload))
        if not quiet:
            print("✅ send:", payload)

        try:
            # 5 秒以内に返事が来なければ TimeoutError
            response = await asyncio.wait_for(ws.recv(), timeout=5)
            if not quiet:
                if indent:
                    print("📨 recv: ", json.dumps(json.loads(response), indent=indent, ensure_ascii=False))
                else:
                    print("📨 recv:", response)
            return json.loads(response)
        except asyncio.TimeoutError:
            print("⏰ タイムアウト（5 秒以内にメッセージが来ませんでした）")

In [21]:

size = 0.05

for i in tqdm(range(100)):

    # 平面の追加
    payload = {
        "type": "add",
        "command": "item",
        "group": 0,
        "category": 0,
        "itemId": 740,
    }
    item_response = await websocket_test(payload, quiet=True)

    # フォルダの追加
    payload = {
        "type": "add",
        "command": "folder",
        "name": "星"
    }
    star_response = await websocket_test(payload, quiet=True)

    # 星空 -> 星
    payload = {
        "type": "hierarchy",
        "command": "attach",
        "parentId": 131,
        "childId": star_response["objectId"],
    }
    response = await websocket_test(payload, quiet=True)

    # 星 -> 星オブジェクト
    payload = {
        "type": "hierarchy",
        "command": "attach",
        "parentId": star_response["objectId"],
        "childId": item_response["objectId"],
    }
    response = await websocket_test(payload, quiet=True)

    # 星オブジェクトの変形
    payload = {
        "type": "update",
        "command": "transform",
        "id": item_response["objectId"],
        "pos": [-20.0, 0.0, 0.0],
        "rot": [0.0, 0.0, 270.0],
        "scale": [size, 0.01, size],
    }
    response = await websocket_test(payload, quiet=True)

    # 星の回転
    payload = {
        "type": "update",
        "command": "transform",
        "id": star_response["objectId"],
        "pos": [0.0, 0.0, 0.0],
        "rot": [
            0.0,
            random.uniform(-17, 17),
            random.uniform(-10, 0),
        ],
    }
    response = await websocket_test(payload, quiet=True)


  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
payload = {
    "type": "tree",
}
tree = await websocket_test(payload, quiet=True)
tree

[{'name': 'フォルダー',
  'objectInfo': {'id': 127,
   'type': 'OCIFolder',
   'transform': {'pos': [3.78557277, -1.54168761, -16.9497643],
    'rot': [0.0, 98.81412, 0.0],
    'scale': [1.0, 1.0, 1.0]}},
  'children': [{'name': 'サーフボード',
    'objectInfo': {'id': 146,
     'type': 'OCIItem',
     'transform': {'pos': [0.22529991, 0.8777963, 0.102002807],
      'rot': [358.294159, 316.9084, 352.9234],
      'scale': [0.5, 0.85, 0.85]},
     'itemDetail': {'group': 3, 'category': 105, 'itemId': 1474}},
    'children': []},
   {'name': '篠崎 遥',
    'objectInfo': {'id': 0,
     'type': 'OCICharFemale',
     'transform': {'pos': [0.0, 0.0, 0.0],
      'rot': [0.0, 0.0, 0.0],
      'scale': [1.0, 1.0, 1.0]}},
    'children': []},
   {'name': '星々',
    'objectInfo': {'id': 131,
     'type': 'OCIFolder',
     'transform': {'pos': [0.0524236932, 1.04399776, 0.0199345611],
      'rot': [351.4012, 247.189667, 355.292328],
      'scale': [1.0, 1.0, 1.0]}},
    'children': [{'name': '星',
      'objectInf

In [5]:
payload = {
    "type": "tree",
    "depth": 1,
}
tree = await websocket_test(payload, quiet=True)
tree

[{'name': 'フォルダー',
  'objectInfo': {'id': 127,
   'type': 'OCIFolder',
   'transform': {'pos': [3.78557277, -1.54168761, -16.9497643],
    'rot': [0.0, 98.81412, 0.0],
    'scale': [1.0, 1.0, 1.0]}},
  'children': [{'name': 'サーフボード',
    'objectInfo': {'id': 146,
     'type': 'OCIItem',
     'transform': {'pos': [0.22529991, 0.8777963, 0.102002807],
      'rot': [358.294159, 316.9084, 352.9234],
      'scale': [0.5, 0.85, 0.85]},
     'itemDetail': {'group': 3, 'category': 105, 'itemId': 1474}},
    'children': []},
   {'name': '篠崎 遥',
    'objectInfo': {'id': 0,
     'type': 'OCICharFemale',
     'transform': {'pos': [0.0, 0.0, 0.0],
      'rot': [0.0, 0.0, 0.0],
      'scale': [1.0, 1.0, 1.0]}},
    'children': []},
   {'name': '星々',
    'objectInfo': {'id': 131,
     'type': 'OCIFolder',
     'transform': {'pos': [0.0524236932, 1.04399776, 0.0199345611],
      'rot': [351.4012, 247.189667, 355.292328],
      'scale': [1.0, 1.0, 1.0]}},
    'children': []}]},
 {'name': 'ポイントライト',
  '

In [6]:
payload = {
    "type": "tree",
    "depth": 1,
    "id": 131,
}
tree = await websocket_test(payload, quiet=True)
tree

{'name': '星々',
 'objectInfo': {'id': 131,
  'type': 'OCIFolder',
  'transform': {'pos': [0.0524236932, 1.04399776, 0.0199345611],
   'rot': [351.4012, 247.189667, 355.292328],
   'scale': [1.0, 1.0, 1.0]}},
 'children': [{'name': '星',
   'objectInfo': {'id': 133,
    'type': 'OCIFolder',
    'transform': {'pos': [0.0, 0.0, 0.0],
     'rot': [0.0, 6.1597743, -5.314398],
     'scale': [1.0, 1.0, 1.0]}},
   'children': []},
  {'name': '星',
   'objectInfo': {'id': 135,
    'type': 'OCIFolder',
    'transform': {'pos': [0.0, 0.0, 0.0],
     'rot': [0.0, -0.743452, -6.59567356],
     'scale': [1.0, 1.0, 1.0]}},
   'children': []},
  {'name': '星',
   'objectInfo': {'id': 137,
    'type': 'OCIFolder',
    'transform': {'pos': [0.0, 0.0, 0.0],
     'rot': [0.0, -1.58394265, -2.71944976],
     'scale': [1.0, 1.0, 1.0]}},
   'children': []},
  {'name': '星',
   'objectInfo': {'id': 139,
    'type': 'OCIFolder',
    'transform': {'pos': [0.0, 0.0, 0.0],
     'rot': [0.0, -12.5233431, -7.20495033],


In [7]:
payload = {
    "type": "tree",
    "id": 135,
}
tree = await websocket_test(payload, quiet=True)
tree

{'name': '星',
 'objectInfo': {'id': 135,
  'type': 'OCIFolder',
  'transform': {'pos': [0.0, 0.0, 0.0],
   'rot': [0.0, -0.743452, -6.59567356],
   'scale': [1.0, 1.0, 1.0]}},
 'children': [{'name': '平面（通常）',
   'objectInfo': {'id': 134,
    'type': 'OCIItem',
    'transform': {'pos': [-20.0, 0.0, 0.0],
     'rot': [0.0, 0.0, 270.0],
     'scale': [0.05, 0.01, 0.05]},
    'itemDetail': {'group': 0, 'category': 0, 'itemId': 740}},
   'children': []}]}